# 年齢(3区分)別の利用者数

利用者のテーブル(users)から、主要な利用者を把握して、サービスの改善に繋げます。
今回は、年齢性別を区分しますが、仮説に応じて様々な観点で分析することが重要です。

In [1]:
%LOAD example.db

In [2]:
SELECT
  *
FROM
  users
LIMIT
  5

id,name,birthdate,sex
1,田中 洋介,1960-06-27,M
2,佐藤 直樹,1961-03-07,M
3,佐藤 英樹,1912-04-24,M
4,田中 知実,1953-03-06,F
5,佐々木 充,1919-03-29,M


## 生年月日から年齢を求める
生年月日を管理している場合は、計算日との差分によって年齢を算出します。

In [3]:
SELECT
  *,
  -- 年齢を求める。SQLite以外では別な求め方になるはずです。
  CAST(STRFTIME('%Y.%m%d', 'now', '+9 hours') - STRFTIME('%Y.%m%d', birthdate) AS INT) AS age
FROM
  users
LIMIT
  5

id,name,birthdate,sex,age
1,田中 洋介,1960-06-27,M,61
2,佐藤 直樹,1961-03-07,M,60
3,佐藤 英樹,1912-04-24,M,109
4,田中 知実,1953-03-06,F,68
5,佐々木 充,1919-03-29,M,102


## 年齢から年齢区分を定める
近い年齢を`CASE`文でグループ名にマッピングして集計しやすくする。

In [4]:
WITH users_with_age AS (
  SELECT
    *,
    -- 年齢を求める。SQLite以外では別な求め方になるはずです。
    CAST(STRFTIME('%Y.%m%d', 'now', '+9 hours') - STRFTIME('%Y.%m%d', birthdate) AS INT) AS age
  FROM
    users
)
SELECT
  *,
  CASE
    WHEN age BETWEEN 0 AND 14 THEN '14歳以下'
    WHEN age BETWEEN 15 AND 64 THEN '15～64歳'
    WHEN 65 <= age THEN '65歳以上'
  END AS age_category
FROM
  users_with_age
LIMIT
  5

id,name,birthdate,sex,age,age_category
1,田中 洋介,1960-06-27,M,61,15～64歳
2,佐藤 直樹,1961-03-07,M,60,15～64歳
3,佐藤 英樹,1912-04-24,M,109,65歳以上
4,田中 知実,1953-03-06,F,68,65歳以上
5,佐々木 充,1919-03-29,M,102,65歳以上


## 年齢区分と性別から人数を定める
年齢区分と性別をグルーピングして、どんな属性の利用者が多いのか特定する。

In [5]:
WITH users_with_age AS (
  SELECT
    *,
    -- 年齢を求める。SQLite以外では別な求め方になるはずです。
    CAST(STRFTIME('%Y.%m%d', 'now', '+9 hours') - STRFTIME('%Y.%m%d', birthdate) AS INT) AS age
  FROM
    users
), users_with_age_category AS (
  SELECT
    *,
    CASE
      WHEN age BETWEEN 0 AND 14 THEN '14歳以下'
      WHEN age BETWEEN 15 AND 64 THEN '15～64歳'
      WHEN 65 <= age THEN '65歳以上'
    END AS age_category
  FROM
    users_with_age
)
SELECT
  age_category,
  sex,
  COUNT(*) AS count
FROM
  users_with_age_category
GROUP BY
  age_category,
  sex
ORDER BY
  age_category,
  sex

age_category,sex,count
14歳以下,F,75
14歳以下,M,58
15～64歳,F,219
15～64歳,M,219
65歳以上,F,217
65歳以上,M,212
